In [ ]:
!pip install stable-baselines3 shimmy gymnasium

  Using cached stable_baselines3-2.7.1-py3-none-any.whl.metadata (4.8 kB)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import re
import math
import random
import warnings
import random
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.sparse import coo_matrix
import gymnasium as gym
from gymnasium import spaces
from sklearn.preprocessing import StandardScaler, LabelEncoder
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import os
warnings.filterwarnings("ignore")

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.use_deterministic_algorithms(True)

set_seed()

# Dataset Overview

Load `dummy_working.xlsx` into the files tab.

In [ ]:
df = pd.read_excel("dummy_working.xlsx")
print("Shape:", df.shape)
df.head()

In [ ]:
df.info()   # gives structural summary of df
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("Numeric columns:", numeric_cols)
print("Categorical/Text columns:", cat_cols)

# Exploratory Data Analysis

## Missing Values

In [ ]:
missing_pct = df.isnull().mean().sort_values(ascending=True) # sorted columns by percentage of missing values
missing_pct[missing_pct > 0]
plt.figure(figsize=(10, 8)) # Height adjusted for column names
missing_pct.plot(kind="barh")
plt.title("Percentage of Missing Values by Column")
plt.xlabel("Missing proportion")
plt.ylabel("Column Name")
plt.grid(axis='x', linestyle='--', alpha=0.7) # Grid on x-axis only
plt.tight_layout() # Ensures labels aren't cut off
plt.show()

## Relationship between `Revenue (USD)` and `Market Value (USD)`

In [ ]:
num_zeros_MV = (df['mark_val_usd'] == 0).sum()
print(f"There are {num_zeros_MV} invalid entries")

df.loc[df['mark_val_usd'] == 0, 'mark_val_usd'] = pd.NA
MV_summary = df['mark_val_usd'].describe()
print(MV_summary)

In [ ]:
# Get only the revenue and market value that is is not NA.
subset = df[['revenue_usd', 'mark_val_usd']].dropna()
X = subset[['revenue_usd']]   # must be 2D
y = subset['mark_val_usd']

# Fit linear regression
model = LinearRegression()
model.fit(X, y)

# Predictions
y_pred = model.predict(X)

# Regression parameters
slope = model.coef_[0]
intercept = model.intercept_
r_squared = model.score(X, y)

# Print regression equation
print(f"Regression equation: Market Value = {slope:.4f} × Revenue + {intercept:.4f}")
print(f"R² = {r_squared:.4f}")

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(X, y)
plt.plot(X, y_pred)
plt.xlabel('Revenue (USD)')
plt.ylabel('Market Value (USD)')
plt.title(f'Market Value vs Revenue')

# Uncomment to see only bottom left part. Comment to see everything
plt.xlim(0, 1e8)   # zoom into revenue < 100 million
plt.ylim(0, 5e8)

plt.show()

## Relationship between `Revenue (USD)` and `IT spend`

In [ ]:
plot_data = df[(df['revenue_usd'] > 0) & (df['it_spending'] > 0)].dropna(subset=['revenue_usd', 'it_spending'])
x = plot_data['it_spending'].values
y = plot_data['revenue_usd'].values

m, c = np.polyfit(x, y, 1)
y_fit = m * x + c
R = np.corrcoef(x, y)[0, 1]

plt.figure(figsize=(12, 7))
plt.scatter(
    x, y,
    alpha=0.5, s=30,
    edgecolors='white', linewidth=0.5
)
plt.plot(
    x, y_fit,
    linewidth=2,
    label=f'Best fit line (R = {R:.3f})'
)
equation_text = f"Revenue = {m:.2f} × IT spend + {c:.2e}\nR = {R:.3f}"

plt.text(
    0.05, 0.95,
    equation_text,
    transform=plt.gca().transAxes,
    fontsize=11,
    verticalalignment='top',
    bbox=dict(boxstyle='round', alpha=0.2)
)

print(f"{equation_text}")

plt.title('Productivity Analysis: Revenue vs IT Spend')
plt.xlabel('IT spend')
plt.ylabel('Revenue (USD)')
plt.grid(True, linestyle='--', alpha=0.3)
plt.legend()

# Uncomment the bottom 2 lines of code to restrict the x axis to 0-0.5e7 and the y axis to 0-0.2e9.
plt.xlim(0, 5e6)
plt.ylim(0, 2e8)

plt.tight_layout()
plt.show()

print(f"Plotting {len(plot_data)} companies.")
print(f"Correlation coefficient (R): {R:.3f}")

From the above plot, we see that there is a positive (linear) relationship between the revenue of the company and IT spending. However, there seems to exist multiple linear relationships between `Revenue` and `IT spend`, each having its own slope.

We can group the above data into 2 categories:
- High efficiency firms (high revenue per dollar of IT)
- Low efficiency firms (similar IT spend, much lower revenue)

In [ ]:
m = 33.60
c = 7.16e5

# Note: efficiency column already exists in dummy_working.xlsx
# Verify the existing classification
plot_data = df[
    (df['revenue_usd'] > 0) &
    (df['it_spending'] > 0)
].dropna(subset=['revenue_usd', 'it_spending', 'efficiency'])

color_map = {
    'High efficiency': 'seagreen',
    'Low efficiency': 'indianred'
}

plt.figure(figsize=(12, 7))

# Scatter plot with colour corresponding to responsiveness of revenue (USD) against IT spend (USD)
for label, color in color_map.items():
    subset = plot_data[plot_data['efficiency'] == label]
    
    plt.scatter(
        subset['it_spending'],
        subset['revenue_usd'],
        alpha=0.6,
        s=30,
        color=color,
        edgecolors='white',
        linewidth=0.5,
        label=label
    )

#Finding out the eqn for the best fit line of revenue against IT spend as well as the R^2 value.
equation_text = f"Revenue = {m:.2f} × IT spend + {c:.2e}\nR^2 = {R:.3f}"

plt.text(
    0.05, 0.95,
    equation_text,
    transform=plt.gca().transAxes,
    fontsize=11,
    verticalalignment='top',
    bbox=dict(boxstyle='round', alpha=0.2)
)


# Best-fit line
x_vals = np.linspace(0, 5e6, 200)
y_vals = m * x_vals + c

plt.plot(
    x_vals,
    y_vals,
    color='black',
    linestyle='--',
    linewidth=2,
    label='Best fit line'
)

plt.title('Productivity Analysis: Revenue vs IT Spend')
plt.xlabel('IT spend (USD)')
plt.ylabel('Revenue (USD)')
plt.grid(True, linestyle='--', alpha=0.3)
plt.legend()

plt.xlim(0, 5e6)
plt.ylim(0, 2e8)

plt.tight_layout()
plt.show()

The green points are companies with a high revenue/IT spend ratio and the red points are companies with a lower revenue/IT spend ratio.

## Relationship between Revenue and Total Employees

In [ ]:
df["revenue_usd"] = pd.to_numeric(df["revenue_usd"], errors="coerce")
df["revenue_usd"] = df["revenue_usd"].replace(0, np.nan)

df.dtypes
df.head()
(df["revenue_usd"] == 0).sum() # Check that the 0's in the revenue are all nan values

In [ ]:
#Check employees total df
df["employees_total"].dtype

df["employees_total"] = df["employees_total"].replace(0, np.nan)
(df["employees_total"] == 0).sum() # replace 0 with nan for companies with 0 employees

In [ ]:
# keep only rows where both values exist
sub = df[["employees_total", "revenue_usd"]].dropna()

plt.figure(figsize=(6,4))
plt.scatter(sub["employees_total"], sub["revenue_usd"])
plt.xlabel("employees_total")
plt.ylabel("revenue_usd")
plt.title("Revenue (USD) vs Total Employees")
plt.ylim (0, 2e8)
plt.show()

In [ ]:
r = sub["employees_total"].corr(sub["revenue_usd"], method="pearson")

plt.figure(figsize=(6,4))
plt.scatter(sub["employees_total"], sub["revenue_usd"])
plt.xlabel("employees_total")
plt.ylabel("revenue_usd")
plt.title(f"Revenue (USD) vs Total Employees (R^2 = {r:.3f})")
plt.show()

In [ ]:
df = df.copy()
df["company_id"] = df.index + 1   # 1,2,3,...

sub = df[["company_id", "employees_total", "revenue_usd"]].dropna().copy()
sub = sub[(sub["employees_total"] > 0) & (sub["revenue_usd"] > 0)]

sub["rev_per_emp"] = sub["revenue_usd"] / sub["employees_total"]

sub.sort_values("rev_per_emp", ascending=False).head(15)

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(sub["employees_total"], sub["rev_per_emp"])
plt.xlabel("Employees Total")
plt.ylabel("Revenue per Employee")
plt.title("Revenue per Employee vs Employees")
plt.show()

# Network Graph

Load `companies_edges.csv` into the files tab.

In [ ]:
# Read data
df = pd.read_excel("dummy_working.xlsx")

# Fill missing values
df.fillna({
    "Id": "Unknown",
    "indegree": 0,
    "outdegree": 0,
    "employees_total": 0,
    "revenue_usd": 0,
    "sic_desc": "NA",
    "corp_fam_members": 0,
    "mark_val_usd": 0,
    "it_spending": 0,
    "efficiency": "Unknown"
}, inplace=True)

# Convert data types
df = df.astype({
    "Id": str,
    "indegree": int,
    "outdegree": int,
    "employees_total": int,
    "revenue_usd": float,
    "sic_desc": str,
    "corp_fam_members": int,
    "mark_val_usd": float,
    "it_spending": float,
    "efficiency": str
})

print(df.head())

In [ ]:
# Read and process company data
df = pd.read_excel("dummy_working.xlsx")
df.fillna({"Id": "Unknown", "indegree": 0, "outdegree": 0,
           "employees_total": 0, "revenue_usd": 0,
           "sic_desc": "NA", "corp_fam_members": 0, "mark_val_usd": 0,
           "it_spending": 0, "efficiency": "Unknown"}, inplace=True)

df = df.astype({"Id": str, "indegree": int, "outdegree": int,
                "employees_total": int, "revenue_usd": float,
                "sic_desc": str, "corp_fam_members": int, "mark_val_usd": float,
                "it_spending": float, "efficiency": str})

edgeList = pd.read_csv("companies_edges.csv", header=None, names=['source', 'target'])

# Convert edge list to a set of tuples for O(1) lookup
edge_set = set(zip(edgeList['source'], edgeList['target']))

# Get list of company IDs
company_ids = df["Id"].tolist()

# Initialise adjacency matrix with False values
adjMat = pd.DataFrame(False, index=company_ids, columns=company_ids)

# Populate adjacency matrix
for source, target in edge_set:
    if target in adjMat.index and source in adjMat.columns:
        adjMat.at[target, source] = True

print(adjMat.head())

# Principal Component Analysis

In [ ]:
df = pd.read_excel("dummy_working.xlsx")

pca_df = df.copy()

# drop columns not suitable for PCA
pca_df = pca_df.drop(columns=["Id", "sic_desc"], errors="ignore")

pca_df.head()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# log-transform skewed business variables (safe with zeros)
skew_cols = ["employees_total","revenue_usd","corp_fam_members","mark_val_usd","it_spending"]
for c in skew_cols:
    if c in pca_df.columns:
        pca_df[c] = pd.to_numeric(pca_df[c], errors="coerce").clip(lower=0)
        pca_df[c] = np.log1p(pca_df[c])

# ensure everything is numeric
X = pca_df.apply(pd.to_numeric, errors="coerce")

# impute missing + scale
X_imp = SimpleImputer(strategy="median").fit_transform(X)
X_scaled = StandardScaler().fit_transform(X_imp)

# PCA
pca = PCA()
Z = pca.fit_transform(X_scaled)

#Calculates how much variance is explained by each successive PC axis.
explained = pca.explained_variance_ratio_
print("Explained variance ratio:", explained.round(4))
print("Cumulative:", explained.cumsum().round(4))

In [ ]:
# % variance per PC
var = pca.explained_variance_ratio_ * 100

plt.figure(figsize=(8,4))
plt.plot(range(1, len(var)+1), var, marker="o")
plt.xlabel("Principal Component")
plt.ylabel("Variance Explained (%)")
plt.title("Scree Plot")
plt.xticks(range(1, len(var)+1, 1))
plt.grid(True)
plt.show()

var = pca.explained_variance_ratio_ * 100  # convert to %

print(f"PC1 variance explained: {var[0]:.2f}%")
print(f"PC2 variance explained: {var[1]:.2f}%")
print(f"PC1+PC2 cumulative: {(var[0]+var[1]):.2f}%")

In [ ]:
# Biplot
# If you already have Z from fit_transform, you can just use it:
pcaX = Z[:, :2]                          # first two principal component SCORES
loadings = pca.components_.T[:, :2]      # (n_features, 2)
feature_names = X.columns                # feature names used in PCA

fig, ax1 = plt.subplots(figsize=(10, 8))  # Create main figure and axis

# Plot data points (scores)
sc = ax1.scatter(
    pcaX[:, 0], pcaX[:, 1],
    s=10, alpha=0.6, linewidth=0.3
)

# Label your primary axis -> PCA scores
ax1.set_xlabel("PC1 scores", fontsize=12, fontweight='bold', labelpad=10)
ax1.set_ylabel("PC2 scores", fontsize=12, fontweight='bold', labelpad=10)
ax1.set_title("PCA Biplot with Score and Loading Axes", fontsize=14, fontweight='bold', pad=25)
ax1.grid(False)

# Secondary axis for loadings (fixed to [-1, 1])
ax2 = ax1.twinx().twiny()
ax2.set_xlim(-1, 1)
ax2.set_ylim(-1, 1)
ax2.set_xlabel("PC1 loadings", fontsize=12, fontweight='bold', color='red', labelpad=10)
ax2.set_ylabel("PC2 loadings", fontsize=12, fontweight='bold', color='red', labelpad=10)
ax2.tick_params(axis='x', colors='red')
ax2.tick_params(axis='y', colors='red')
ax2.grid(False)

# Draw loading arrows.
# Your scale formula is okay; it makes arrows visible relative to score spread.
scale = (np.max(np.abs(pcaX)) / np.max(np.abs(loadings))) * 0.5

for feature, (lx, ly) in zip(feature_names, loadings[:, :2]):
    ax1.arrow(
        0, 0,
        lx * scale, ly * scale,
        color='red', alpha=0.7,
        head_width=0.1,
        length_includes_head=True
    )
    ax1.text(
        lx * scale * 1.1, ly * scale * 1.1,
        feature,
        color='red',
        ha='center', va='center',
        fontsize=10,
        fontweight='bold'
    )

# Reference axis lines
ax1.axhline(0, color="gray", linestyle="--", linewidth=1)
ax1.axvline(0, color="gray", linestyle="--", linewidth=1)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# -----------------------------
# Helper: biplot PC1 vs PC2
# -----------------------------
def biplot_pc1_pc2(Z, loadings, feature_names, title, df, top_n=10):
    scores = Z[:, :2]
    load2 = loadings[:, :2]
    
    # efficiency mask + values
    mask = df["efficiency"].notna().values
    eff = df.loc[mask, "efficiency"].map({
        "Low efficiency": 0,
        "High efficiency": 1
    }).values
    
    # choose top_n arrows by length (importance in PC1/PC2 plane)
    strength = np.sqrt(load2[:, 0]**2 + load2[:, 1]**2)
    top_idx = np.argsort(strength)[-top_n:]
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # scatter scores by efficiency
    ax.scatter(
        scores[mask][eff == 0, 0],
        scores[mask][eff == 0, 1],
        s=8, alpha=0.35, linewidth=0,
        label="Low efficiency"
    )
    
    ax.scatter(
        scores[mask][eff == 1, 0],
        scores[mask][eff == 1, 1],
        s=8, alpha=0.35, linewidth=0,
        label="High efficiency"
    )
    
    ax.set_xlabel("PC1 scores", fontweight="bold")
    ax.set_ylabel("PC2 scores", fontweight="bold")
    ax.set_title(title, fontweight="bold")
    ax.axhline(0, color="gray", linestyle="--", linewidth=1)
    ax.axvline(0, color="gray", linestyle="--", linewidth=1)
    ax.grid(False)
    
    # scale arrows to match the score spread
    scale = (np.max(np.abs(scores)) / np.max(np.abs(load2[top_idx]))) * 0.45
    
    # draw arrows + labels
    for i in top_idx:
        x, y = load2[i, 0] * scale, load2[i, 1] * scale
        ax.arrow(0, 0, x, y, alpha=0.75, head_width=0.06, length_includes_head=True)
        ax.text(
            x * 1.10, y * 1.10, feature_names[i],
            fontsize=9, fontweight="bold",
            ha="center", va="center"
        )
    
    ax.legend(frameon=False)
    plt.tight_layout()
    plt.show()

biplot_pc1_pc2(
    Z, pca.components_.T, X.columns,
    title="PCA Biplot (PC1 vs PC2) - Colored by Efficiency",
    df=df,
    top_n=min(8, len(X.columns))
)

# XGBoost and Random Forest Models

## Training Data

In [ ]:
df = pd.read_excel("dummy_working.xlsx")

df.fillna({"Id": "Unknown", "indegree": 0, "outdegree": 0,
           "employees_total": 0, "revenue_usd": 0,
           "sic_desc": "NA", "corp_fam_members": 0, "mark_val_usd": 0,
           "it_spending": 0, "efficiency": "Unknown"}, inplace=True)

df = df.astype({"Id": str, "indegree": int, "outdegree": int,
                "employees_total": int, "revenue_usd": float,
                "sic_desc": str, "corp_fam_members": int, "mark_val_usd": float,
                "it_spending": float, "efficiency": str})

print(f"Original dataframe shape: {df.shape}")
if df.duplicated(subset=['Id']).any():
    print(f"WARNING: Found {df.duplicated(subset=['Id']).sum()} duplicate company IDs")
    df = df.drop_duplicates(subset=['Id'], keep='first')
    print(f"After removing duplicates: {df.shape}")

le = LabelEncoder()
df['sic_desc_encoded'] = le.fit_transform(df['sic_desc'])

df = df.set_index("Id")

edgeList = pd.read_csv("companies_edges.csv", header=None, names=['source', 'target'])
edge_set = set(zip(edgeList['source'], edgeList['target']))

print(f"Total companies: {len(df)}")
print(f"Total edges: {len(edge_set)}")

In [ ]:
def create_pairwise_dataset_balanced(attributes, edge_set, negative_multiplier=2):
    # Select only numeric columns
    features_to_use = attributes.select_dtypes(include=[np.number]).columns.tolist()
    print(f"Using {len(features_to_use)} numeric features")
    
    # Convert to numpy for fast access
    company_ids = attributes.index.tolist()
    company_to_idx = {cid: idx for idx, cid in enumerate(company_ids)}
    attributes_array = attributes[features_to_use].values
    
    # Separate positive and negative pairs
    positive_pairs = []
    all_companies_set = set(company_ids)
    
    print("Processing positive pairs (actual acquisitions)...")
    for source, target in edge_set:
        if source in all_companies_set and target in all_companies_set:
            parent_idx = company_to_idx[target]
            child_idx = company_to_idx[source]
            
            parent_attrs = attributes_array[parent_idx]
            child_attrs = attributes_array[child_idx]
            
            combined = np.concatenate([
                parent_attrs,
                child_attrs,
                parent_attrs - child_attrs,
                parent_attrs / (child_attrs + 1e-8),
            ])
            
            positive_pairs.append(combined)
    
    print(f"Found {len(positive_pairs)} positive pairs")
    
    # Sample negative pairs
    num_negatives_needed = len(positive_pairs) * negative_multiplier
    print(f"Sampling {num_negatives_needed} negative pairs...")
    
    negative_pairs = []
    edge_dict = {(target, source) for source, target in edge_set}
    
    np.random.seed(42)
    attempts = 0
    max_attempts = num_negatives_needed * 10  # Safety limit
    
    while len(negative_pairs) < num_negatives_needed and attempts < max_attempts:
        # Randomly sample two different companies
        parent_id = np.random.choice(company_ids)
        child_id = np.random.choice(company_ids)
        
        # Skip if same company or if this is actually a positive pair
        if parent_id == child_id or (parent_id, child_id) in edge_dict:
            attempts += 1
            continue
        
        parent_idx = company_to_idx[parent_id]
        child_idx = company_to_idx[child_id]
        
        parent_attrs = attributes_array[parent_idx]
        child_attrs = attributes_array[child_idx]
        
        combined = np.concatenate([
            parent_attrs,
            child_attrs,
            parent_attrs - child_attrs,
            parent_attrs / (child_attrs + 1e-8),
        ])
        
        negative_pairs.append(combined)
        attempts += 1
        
        if len(negative_pairs) % 500 == 0:
            print(f"  Sampled {len(negative_pairs)}/{num_negatives_needed} negative pairs...")
    
    # Combine positive and negative
    X = np.vstack([positive_pairs, negative_pairs])
    y = np.array([1] * len(positive_pairs) + [0] * len(negative_pairs))
    
    print(f"\nFinal dataset:")
    print(f"  Total pairs: {len(y):,}")
    print(f"  Positive: {sum(y):,} ({sum(y)/len(y)*100:.1f}%)")
    print(f"  Negative: {len(y) - sum(y):,} ({(len(y)-sum(y))/len(y)*100:.1f}%)")
    print(f"  Feature vector size: {X.shape[1]}")
    
    return X, y

In [ ]:
# Create balanced dataset
X, y = create_pairwise_dataset_balanced(df, edge_set, negative_multiplier=2)
# negative_multiplier means 2 negative samples per positive sample

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"\nTraining set: {len(X_train):,} samples")
print(f"Test set: {len(X_test):,} samples")

## XGBoost Classifier

In [ ]:
ratio = (len(y_train) - sum(y_train)) / sum(y_train)

param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'min_child_weight': [1, 5, 10],
    'gamma': [0, 0.1, 0.2], # Regularization: helps prune branches that don't help
    'n_estimators': [100, 200, 300]
}

xgb_base = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=ratio, # Keep the ratio we calculated earlier
    use_label_encoder=False,
    eval_metric='auc',
    random_state=42
)

random_search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_grid,
    n_iter=20, # n_iter=20 means it will try 20 different random combinations
    scoring='roc_auc',
    cv=5, # 5-fold cross validation
    verbose=2,
    random_state=42,
    n_jobs=-1 # Use all CPU cores
)

print("Starting Parameter Search...")
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

print(f"\nBest Parameters: {random_search.best_params_}")
print(f"Best CV Score (AUC): {random_search.best_score_:.4f}")

feature_names = []
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
for prefix in ['parent_', 'child_', 'diff_', 'ratio_']:
    feature_names.extend([prefix + f for f in numeric_features])

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

## Random Forest Model

In [ ]:
print("Training Random Forest Model...")
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

def evaluate(model, name):
    probs = model.predict_proba(X_test)[:, 1]
    preds = model.predict(X_test)
    print(f"\n--- {name} Results ---")
    print(f"ROC-AUC: {roc_auc_score(y_test, probs):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
    print(classification_report(y_test, preds, target_names=['No Deal', 'Acquisition']))

evaluate(rf_model, "Random Forest")

rf_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(rf_importance.head(10))

# Reinforcement Learning Model

In [ ]:
# 1. Data Loading and Preprocessing
def load_and_preprocess_data(nodes_path, edges_path):
    if not os.path.exists(nodes_path) or not os.path.exists(edges_path):
        raise FileNotFoundError("Check your file paths; one or both files are missing.")
    
    print(f"Loading nodes from {nodes_path}...")
    # Load Excel file (ensure openpyxl is installed)
    nodes_df = pd.read_excel(nodes_path)
    
    # --- RECTIFICATION FOR DUPLICATE INDEX ERROR ---
    initial_count = len(nodes_df)
    nodes_df = nodes_df.drop_duplicates(subset=['Id'], keep='first')
    final_count = len(nodes_df)
    if initial_count > final_count:
        print(f"Removed {initial_count - final_count} duplicate company IDs.")
    # -----------------------------------------------
    
    print(f"Loading edges from {edges_path}...")
    # Read CSV with latin1 to handle special characters from Excel exports
    edges_df = pd.read_csv(edges_path, encoding='latin1')
    
    # Preprocess nodes (Encoding Categoricals)
    le_sic = LabelEncoder()
    nodes_df['sic_desc'] = le_sic.fit_transform(nodes_df['sic_desc'].astype(str))
    
    le_eff = LabelEncoder()
    nodes_df['efficiency'] = le_eff.fit_transform(nodes_df['efficiency'].astype(str))
    
    # Scale numerical features (Essential for RL convergence)
    num_cols = ['indegree', 'outdegree', 'employees_total', 'revenue_usd',
                'corp_fam_members', 'mark_val_usd', 'it_spending']
    
    scaler = StandardScaler()
    nodes_df[num_cols] = scaler.fit_transform(nodes_df[num_cols])
    
    # Prepare feature lookup
    feature_cols = num_cols + ['sic_desc', 'efficiency']
    node_features = nodes_df.set_index('Id')[feature_cols].to_dict('index')
    
    # Prepare label lookup (Adjacency Set)
    adj_set = set(zip(edges_df['source'].astype(str), edges_df['target'].astype(str)))
    company_ids = nodes_df['Id'].astype(str).tolist()
    
    return node_features, adj_set, company_ids, len(feature_cols)

# 2. Custom Gymnasium Environment
class CompanyAcquisitionEnv(gym.Env):
    def __init__(self, node_features, adj_set, company_ids, feature_dim):
        super(CompanyAcquisitionEnv, self).__init__()
        self.node_features = node_features
        self.adj_set = adj_set
        self.company_ids = company_ids
        self.feature_dim = feature_dim
        
        # Action: 0 (No), 1 (Yes)
        self.action_space = spaces.Discrete(2)
        
        # State: Concatenated Acquirer + Target features
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(2 * self.feature_dim,), dtype=np.float32
        )
        
        self.current_acquirer = None
        self.current_target = None
        self.steps_per_episode = 100
        self.current_step = 0
    
    def _get_obs(self):
        # Retrieve features from dictionaries
        acq_feat = np.array(list(self.node_features[self.current_acquirer].values()))
        trg_feat = np.array(list(self.node_features[self.current_target].values()))
        return np.concatenate([acq_feat, trg_feat]).astype(np.float32)
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        
        # Pick random companies for the start of the episode
        self.current_acquirer = np.random.choice(self.company_ids)
        self.current_target = np.random.choice(self.company_ids)
        
        # Prevent self-acquisition
        while self.current_target == self.current_acquirer:
            self.current_target = np.random.choice(self.company_ids)
        
        return self._get_obs(), {}
    
    def step(self, action):
        is_true_acquisition = (self.current_acquirer, self.current_target) in self.adj_set
        
        if action == 1: # Predict "Acquire"
            if is_true_acquisition:
                reward = 10.0  # High reward for finding the needle in the haystack
            else:
                reward = -1.0  # Penalty for false positive
        else: # Predict "Skip"
            if is_true_acquisition:
                reward = -5.0  # Significant penalty for missing a real opportunity
            else:
                reward = 0.1   # Small reward for correct rejection
        
        self.current_step += 1
        
        # Transition to next random pair
        self.current_acquirer = np.random.choice(self.company_ids)
        self.current_target = np.random.choice(self.company_ids)
        while self.current_target == self.current_acquirer:
            self.current_target = np.random.choice(self.company_ids)
        
        terminated = self.current_step >= self.steps_per_episode
        return self._get_obs(), reward, terminated, False, {}

# Main Execution
if __name__ == "__main__":
    NODES_FILE = 'dummy_working.xlsx'
    EDGES_FILE = 'companies_edges.csv'
    
    try:
        print("Initializing data processing...")
        node_feats, adj_set, ids, f_dim = load_and_preprocess_data(NODES_FILE, EDGES_FILE)
        
        # Instantiate Environment
        env = CompanyAcquisitionEnv(node_feats, adj_set, ids, f_dim)
        check_env(env)
        
        # Setup and Train PPO Model
        print("Training model...")
        model = PPO("MlpPolicy", env, verbose=1)
        model.learn(total_timesteps=30000)
        
        # Save Model
        model.save("acquisition_model_final")
        print("\nSuccess! Model saved as 'acquisition_model_final.zip'")
    
    except Exception as e:
        print(f"Process failed: {str(e)}")

# Graph Neural Network Model

In [ ]:
# 1. Load Data
nodes_df = pd.read_excel('dummy_working.xlsx')
edges_df = pd.read_csv('companies_edges.csv')

# This ensures no duplicates
nodes_df = nodes_df.drop_duplicates(subset=['Id']).reset_index(drop=True)

num_cols = ['indegree', 'outdegree', 'employees_total', 'revenue_usd',
            'corp_fam_members', 'mark_val_usd', 'it_spending']
scaler = StandardScaler()
nodes_df[num_cols] = scaler.fit_transform(nodes_df[num_cols])

sic_dummies = pd.get_dummies(nodes_df['sic_desc'], prefix='sic')
eff_dummies = pd.get_dummies(nodes_df['efficiency'], prefix='eff')

# Combine features and store column names for the interface
X_df = pd.concat([nodes_df[num_cols], sic_dummies, eff_dummies], axis=1).astype(float)
feature_cols = X_df.columns.tolist()
X = torch.tensor(X_df.values, dtype=torch.float)

# Mapping node IDs to indices
node_to_idx = {name: i for i, name in enumerate(nodes_df['Id'])}
idx_to_node = {i: name for name, i in node_to_idx.items()}
num_nodes = len(node_to_idx)

# Filter edges to only include nodes present in the cleaned nodes list
valid_edges = edges_df[edges_df['source'].isin(node_to_idx) &
                      edges_df['target'].isin(node_to_idx)]
edge_indices = [[node_to_idx[r['source']], node_to_idx[r['target']]] for _, r in valid_edges.iterrows()]
edge_index = torch.tensor(edge_indices, dtype=torch.long).t()

# 2. Define Model Structure
class GCNLinkPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNLinkPredictor, self).__init__()
        self.lin1 = nn.Linear(in_channels, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, out_channels)
    
    def forward(self, x, adj):
        # Sparse Matrix Multiplication for GCN layer
        x = F.relu(self.lin1(torch.spmm(adj, x)))
        x = self.lin2(torch.spmm(adj, x))
        return x

def get_adj_norm(edge_index, num_nodes):
    row, col = edge_index.numpy()
    adj = coo_matrix((np.ones(row.shape[0]), (row, col)), shape=(num_nodes, num_nodes))
    adj = (adj + adj.T).tocoo()
    adj.setdiag(1)
    d_inv_sqrt = np.power(np.array(adj.sum(1)), -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat = coo_matrix((d_inv_sqrt, (np.arange(num_nodes), np.arange(num_nodes))))
    adj_norm = d_mat.dot(adj).dot(d_mat).tocoo()
    indices = torch.from_numpy(np.vstack((adj_norm.row, adj_norm.col)).astype(np.int64))
    return torch.sparse_coo_tensor(indices, torch.from_numpy(adj_norm.data.astype(np.float32)), [num_nodes, num_nodes])

# 3. Train the Model
adj = get_adj_norm(edge_index, num_nodes)
model = GCNLinkPredictor(X.shape[1], 32, 16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(101):
    model.train()
    optimizer.zero_grad()
    z = model(X, adj)
    pos_scores = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)
    neg_idx = torch.randint(0, num_nodes, (2, edge_index.shape[1]))
    neg_scores = (z[neg_idx[0]] * z[neg_idx[1]]).sum(dim=-1)
    loss = -torch.log(torch.sigmoid(pos_scores) + 1e-15).mean() - \
           torch.log(1 - torch.sigmoid(neg_scores) + 1e-15).mean()
    loss.backward()
    optimizer.step()

# 4. Generate Model Files
torch.save(model.state_dict(), 'gnn_acquisition_model.pth')
metadata = {
    'node_to_idx': node_to_idx,
    'idx_to_node': idx_to_node,
    'num_cols': num_cols,
    'feature_cols': feature_cols,
    'scaler': scaler,
    'in_channels': X.shape[1],
    'hidden_channels': 32,
    'out_channels': 16
}
with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print("Saved: gnn_acquisition_model.pth and model_metadata.pkl")

from sklearn.metrics import roc_auc_score, average_precision_score

def evaluate(model, X, adj, pos_edges, neg_edges):
    model.eval()
    with torch.no_grad():
        z = model(X, adj)
        
        pos_scores = (z[pos_edges[0]] * z[pos_edges[1]]).sum(dim=-1)
        neg_scores = (z[neg_edges[0]] * z[neg_edges[1]]).sum(dim=-1)
        
        scores = torch.cat([pos_scores, neg_scores]).cpu().numpy()
        labels = np.hstack([
            np.ones(len(pos_scores)),
            np.zeros(len(neg_scores))
        ])
        
        probs = torch.sigmoid(torch.tensor(scores)).numpy()
        
        auc = roc_auc_score(labels, probs)
        ap = average_precision_score(labels, probs)
    
    return auc, ap

def split_edges(edge_index, val_ratio=0.1, test_ratio=0.1):
    num_edges = edge_index.shape[1]
    perm = torch.randperm(num_edges)
    
    test_size = int(num_edges * test_ratio)
    val_size = int(num_edges * val_ratio)
    
    test_edges = edge_index[:, perm[:test_size]]
    val_edges = edge_index[:, perm[test_size:test_size+val_size]]
    train_edges = edge_index[:, perm[test_size+val_size:]]
    
    return train_edges, val_edges, test_edges

def negative_sampling(num_nodes, num_samples):
    return torch.randint(0, num_nodes, (2, num_samples))

train_e, val_e, test_e = split_edges(edge_index)

val_neg = negative_sampling(num_nodes, val_e.shape[1])
test_neg = negative_sampling(num_nodes, test_e.shape[1])

val_auc, val_ap = evaluate(model, X, adj, val_e, val_neg)
test_auc, test_ap = evaluate(model, X, adj, test_e, test_neg)

print(f"Validation AUC: {val_auc:.4f}, AP: {val_ap:.4f}")
print(f"Test AUC: {test_auc:.4f}, AP: {test_ap:.4f}")